In [2]:
import os
import psycopg2
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv(".env")

# Obtenir les informations d'identification de la base de données depuis le fichier .env
USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")


def open_connection():
    try:
        conn = psycopg2.connect(
            user=USER_PSQL,
            password=PASSWORD_PSQL,
            host="localhost",
            port="5432",
            dbname="app_auth"
        )
        return conn
    except Exception as e:
        print(f"Erreur de connexion à la base de données: {e}")
        return None


def close_connection(conn):
    if conn is not None:
        conn.close()


## Question 1: Création de la Table `session_count` et du Trigger `trigger_update_session_count`

Nous allons créer une table `session_count` et un trigger `trigger_update_session_count`. Ce trigger mettra à jour le compteur de sessions (`session_count_value`) pour chaque utilisateur après chaque insertion ou suppression dans la table `session_table`.

In [5]:
conn = open_connection()
cursor = conn.cursor()

create_table_script = """
CREATE TABLE IF NOT EXISTS username_history_from_app (
    history_id SERIAL PRIMARY KEY,
    user_id INT REFERENCES user_table (user_id),
    username_new VARCHAR(255)
);
"""
cursor.execute(create_table_script)
conn.commit()

close_connection(conn)

## Comparaison des Temps d'Exécution pour la Mise à Jour de Username

Nous comparerons les temps d'exécution nécessaires pour modifier le `username` en utilisant un trigger et en enregistrant directement l'historique via le code Python.

In [7]:
import time

# Supposons que nous utilisons un user_id existant pour le test
user_id = 1  

def update_username_and_measure_time(conn, user_id, new_username, table_name):
    start_time = time.time()
    cursor = conn.cursor()
    update_query = f"UPDATE user_table SET username = %s WHERE user_id = %s"
    cursor.execute(update_query, (new_username, user_id))
    if table_name == "username_history_from_app":
        insert_query = f"INSERT INTO {table_name} (user_id, username_new) VALUES (%s, %s)"
        cursor.execute(insert_query, (user_id, new_username))
    conn.commit()
    end_time = time.time()
    return end_time - start_time

# Test pour 100, 1000, 10000 éditions
for num_edits in [100, 1000, 10000]:
    conn = open_connection()

    # Avec Trigger
    time_with_trigger = sum(update_username_and_measure_time(conn, user_id, f"new_username_{i}", "username_history") for i in range(num_edits))

    # Sans Trigger (Directement dans username_history_from_app)
    time_without_trigger = sum(update_username_and_measure_time(conn, user_id, f"new_username_{i}", "username_history_from_app") for i in range(num_edits))

    close_connection(conn)

    print(f"Nombre d'éditions: {num_edits}")
    print(f"Temps avec trigger: {time_with_trigger} secondes")
    print(f"Temps sans trigger: {time_without_trigger} secondes")


Nombre d'éditions: 100
Temps avec trigger: 0.07471728324890137 secondes
Temps sans trigger: 0.10446810722351074 secondes
Nombre d'éditions: 1000
Temps avec trigger: 0.6452832221984863 secondes
Temps sans trigger: 0.8157823085784912 secondes
Nombre d'éditions: 10000
Temps avec trigger: 5.323859930038452 secondes
Temps sans trigger: 7.273778915405273 secondes
